In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


# Generate data

In [2]:
training_data = get_pendulum_data(100, True)
validation_data = get_pendulum_data(10)

Generating pendulum data with noise
Generating pendulum data


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [10]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-05-27 16:54:22.657186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-27 16:54:22.657711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-05-27 16:54:22.824966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-27 16:54:23.354672: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-27 16:54:30.579353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 0
   training loss 0.023329727351665497, (0.008067376, 0.0012238977, 30.504637, 0.99707335)
   validation loss 0.01762949489057064, (0.007982797, 0.0010961044, 19.273344, 0.99707335)


2024-05-27 16:54:31.664774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


decoder loss ratio: 0.951718, decoder SINDy loss  ratio: 1.000000
Epoch 100
   training loss 0.02279154397547245, (0.0075332723, 0.00011624728, 30.504637, 0.5946864)
   validation loss 0.017081409692764282, (0.007438786, 8.30812e-05, 19.273344, 0.5946864)
decoder loss ratio: 0.886860, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.022816041484475136, (0.007562197, 0.0027833758, 30.504637, 0.13850556)
   validation loss 0.017071908339858055, (0.0074337577, 0.0018736854, 19.273342, 0.13850556)
decoder loss ratio: 0.886261, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.022823935374617577, (0.0075696846, 0.033020917, 30.504637, 0.028064948)
   validation loss 0.01706969179213047, (0.00743178, 0.01915347, 19.273344, 0.028064948)
decoder loss ratio: 0.886025, decoder SINDy loss  ratio: 1.000000
Epoch 400
   training loss 0.02191402018070221, (0.0065866266, 1.5434088, 30.498857, 0.079308204)
   validation loss 0.01634349301457405, (0.0066517624, 1.1212306,

2024-05-27 17:19:31.316440: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 600
   training loss 0.012782356701791286, (0.0046877107, 2.7292242, 15.915383, 0.049242288)
   validation loss 0.00907565001398325, (0.0039584576, 2.1460958, 10.01879, 0.049242288)
decoder loss ratio: 0.471932, decoder SINDy loss  ratio: 0.519826
Epoch 700
   training loss 0.00846403744071722, (0.0036927569, 1.9253316, 9.349077, 0.04745991)
   validation loss 0.0061425017192959785, (0.0029439263, 1.4543377, 6.2507677, 0.04745991)
decoder loss ratio: 0.350978, decoder SINDy loss  ratio: 0.324322
Epoch 800
   training loss 0.005846149288117886, (0.0029304433, 1.4074395, 5.68976, 0.045347136)
   validation loss 0.0044002896174788475, (0.002335437, 1.1259023, 4.016207, 0.045347136)
decoder loss ratio: 0.278433, decoder SINDy loss  ratio: 0.208381
Epoch 900
   training loss 0.004180005751550198, (0.0023357412, 1.0274268, 3.584962, 0.04121992)
   validation loss 0.003280152101069689, (0.0019113913, 0.8458982, 2.6521075, 0.04121992)
decoder loss ratio: 0.227878, decoder SINDy loss  rat